In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2232467089428390770, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9221160305
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1899044487927774661
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6683898676
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2024142322383381888
 physical_device_desc: "device: 1, name: GeForce GTX 1070 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time

import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
import keras.backend as K
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.1.6
tensorflow Version 1.10.0
dim_ordering: tf


In [5]:
output_classes = 2
batch_size = 16
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 59168
nb_validation_samples = 18384
nb_test_samples = 967

In [6]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 59168 images belonging to 2 classes.
Found 18384 images belonging to 2 classes.
Found 967 images belonging to 2 classes.


In [8]:
def ensembel_model():
    NasNetLarge_model = load_model('models/5.NASNetLarge-new-ISBI19-Model.h5', compile=False)
    NasNetLarge_model = Model(inputs= NasNetLarge_model.input,outputs = NasNetLarge_model.get_layer('dense_3').output, name='NasNetLarge_model')
    VGG19_model = load_model('models/11.VGG19-ISBI19-Model.h5')
    VGG19_model = Model(inputs = VGG19_model.input,outputs = VGG19_model.get_layer('dense_3').output, name='VGG19_model')
    
    img = Input(shape=(224, 224,3),name='img')

    feature1=NasNetLarge_model(img)
    # feature2=InceptionV3_model(img)

    for layer in NasNetLarge_model.layers[:300]:  
        layer.trainable = False 
    for layer in NasNetLarge_model.layers[300:]:  
        layer.trainable = True  

    feature3=VGG19_model(img)
    for layer in VGG19_model.layers[:170]:  
        layer.trainable = False 
    for layer in VGG19_model.layers[170:]:  
        layer.trainable = True  

    # x = concatenate([feature1,feature2,feature3])
    x = concatenate([feature1,feature3])

    x = Dropout(0.5)(x)
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.25)(x)
    output = Dense(2, activation='softmax', name='output')(x)

    model=Model(inputs=img,outputs=output)


    return model

model = ensembel_model()

In [9]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
         'decay':hp.choice('decay',[0.0, 0.004, 0.0001, 0.1, 0.3, 0.5]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [10]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

   
#     adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    adam_opt = Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=1e-08, decay=params['decay'])
    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [11]:
# def get_callbacks(params):
#     callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
#                 ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
#              TensorBoard('tensorlogs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
#     return callbacks

In [12]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.3, 'beta_2': 0.999, 'decay': 0.3, 'lr': 0.001}
Epoch 1/15
3698/3698 [==============================] - 2330s 630ms/step - loss: 0.7063 - acc: 0.5960 - val_loss: 0.6493 - val_acc: 0.8596
Epoch 2/15
3698/3698 [==============================] - 2277s 616ms/step - loss: 0.6474 - acc: 0.6682 - val_loss: 0.6222 - val_acc: 0.9380
Epoch 3/15
3698/3698 [==============================] - 2276s 615ms/step - loss: 0.6271 - acc: 0.7008 - val_loss: 0.6060 - val_acc: 0.9409
Epoch 4/15
3698/3698 [==============================] - 2278s 616ms/step - loss: 0.6151 - acc: 0.7175 - val_loss: 0.5954 - val_acc: 0.9433
Epoch 5/15
3698/3698 [==============================] - 2279s 616ms/step - loss: 0.6055 - acc: 0.7302 - val_loss: 0.5871 - val_acc: 0.9440
Epoch 6/15
3698/3698 [==============================] - 2283s 617ms/step - loss: 0.5974 - acc: 0.7431 - val_loss: 0.5805 - val_acc: 0.9453
Epoch 7/15
3698/3698 [==============================] - 2281s 617ms/step - loss: 0.59

3698/3698 [==============================] - 2309s 624ms/step - loss: 0.2765 - acc: 0.8936 - val_loss: 0.3690 - val_acc: 0.9520
Epoch 7/15
3698/3698 [==============================] - 2310s 625ms/step - loss: 0.2761 - acc: 0.8944 - val_loss: 0.3677 - val_acc: 0.9525
Epoch 8/15
3698/3698 [==============================] - 2309s 624ms/step - loss: 0.2695 - acc: 0.8977 - val_loss: 0.3809 - val_acc: 0.9518
Epoch 9/15
3698/3698 [==============================] - 2309s 624ms/step - loss: 0.2672 - acc: 0.8982 - val_loss: 0.3688 - val_acc: 0.9525
Epoch 10/15
3698/3698 [==============================] - 2308s 624ms/step - loss: 0.2725 - acc: 0.8958 - val_loss: 0.3820 - val_acc: 0.9518
Epoch 11/15
3698/3698 [==============================] - 2304s 623ms/step - loss: 0.2806 - acc: 0.8952 - val_loss: 0.3889 - val_acc: 0.9521
Epoch 12/15
3698/3698 [==============================] - 2307s 624ms/step - loss: 0.2789 - acc: 0.8957 - val_loss: 0.3909 - val_acc: 0.9519
Epoch 13/15
3698/3698 [============

ResourceExhaustedError: OOM when allocating tensor with shape[16,14,14,336] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_7/Adam/gradients/zeros_357-0-1-TransposeNCHWToNHWC-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_7/Adam/gradients/zeros_357, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: output/BiasAdd/_51005 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_44717_output/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_al